<a href="https://colab.research.google.com/github/HemanthSanju/eCommerce_Data_Analysis/blob/Lookalike_Model/Hemanth_Pilli_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
customers_df = pd.read_csv('/content/Customers.csv')
product_df = pd.read_csv('/content/Products.csv')
transactions_df = pd.read_csv('/content/Transactions.csv')

In [3]:
transactions = pd.merge(transactions_df, customers_df, on='CustomerID')
transactions = pd.merge(transactions, product_df, on='ProductID')

In [4]:
customer_product_matrix = transactions_df.pivot_table(index='CustomerID', columns='ProductID', values='Quantity', fill_value=0)


In [5]:
similarity_cf = cosine_similarity(customer_product_matrix)


In [6]:
customer_features = customers_df[['Region', 'SignupDate']]
customer_features['Region'] = customer_features['Region'].astype('category')
customer_features['SignupDate'] = pd.to_datetime(customer_features['SignupDate'])

<ipython-input-6-36f156f46eaf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_features['Region'] = customer_features['Region'].astype('category')
<ipython-input-6-36f156f46eaf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_features['SignupDate'] = pd.to_datetime(customer_features['SignupDate'])


In [7]:
vectorizer = TfidfVectorizer()
customer_features_vectorized = vectorizer.fit_transform(customer_features['Region'].astype(str) + ' ' + customer_features['SignupDate'].dt.strftime('%Y-%m-%d'))
similarity_cbf = cosine_similarity(customer_features_vectorized)



In [8]:
similarity_cbf = similarity_cbf[:-1, :-1]
similarity_hybrid = 0.5 * similarity_cf + 0.5 * similarity_cbf

In [9]:
lookalikes = []
for i in range(len(similarity_hybrid)):
    customer_id = customers_df.iloc[i]['CustomerID']
    scores = similarity_hybrid[i]
    top_3_lookalikes = sorted(zip(customers_df['CustomerID'], scores), key=lambda x: x[1], reverse=True)[1:4]
    lookalikes.append((customer_id, top_3_lookalikes))

In [10]:
lookalikes_df = pd.DataFrame(lookalikes, columns=['CustomerID', 'Lookalikes'])
lookalikes_df.to_csv('Lookalike.csv', index=False)


In [11]:
print("Lookalike recommendations saved to 'Lookalike.csv'")

Lookalike recommendations saved to 'Lookalike.csv'


In [12]:
customers_df

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15
...,...,...,...,...
195,C0196,Laura Watts,Europe,2022-06-07
196,C0197,Christina Harvey,Europe,2023-03-21
197,C0198,Rebecca Ray,Europe,2022-02-27
198,C0199,Andrea Jenkins,Europe,2022-12-03


In [13]:
lookalike_df=pd.read_csv("/content/Lookalike.csv")

In [14]:
lookalike_df

,CustomerID,Lookalikes
0,C0001,"[('C0065', 0.5642546331515212), ('C0104', 0.54..."
1,C0002,"[('C0079', 0.40932392882883895), ('C0166', 0.4..."
2,C0003,"[('C0067', 0.369023811307146), ('C0191', 0.354..."
3,C0004,"[('C0001', 0.3564573321702571), ('C0104', 0.35..."
4,C0005,"[('C0096', 0.3813181084605913), ('C0159', 0.31..."
...,...,...
194,C0195,"[('C0058', 0.43464262896247935), ('C0113', 0.4..."
195,C0196,"[('C0069', 0.32725551848001144), ('C0075', 0.3..."
196,C0197,"[('C0149', 0.4434597348194042), ('C0199', 0.38..."
197,C0198,"[('C0104', 0.4504159205441439), ('C0089', 0.40..."
